<a href="https://colab.research.google.com/github/AdarshShah/IMDB-movie-reviews/blob/master/IMDB_Movie_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow_datasets as tfds
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Embedding, Input, Conv2D, LeakyReLU,  Activation, Reshape
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data = tfds.load("imdb_reviews")

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [0]:
train_data = data['train']
X_train = []
Y_train = []
for features in train_data:
  X_train.append(str(features['text']))
  Y_train.append(int(features['label']))

In [0]:
test_data = data['test']
X_test = []
Y_test = []
for features in test_data:
  X_test.append(str(features['text']))
  Y_test.append(int(features['label']))

In [0]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train,maxlen=2527,padding='post')

In [0]:
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test),maxlen=2527,padding='post')

In [0]:
tf.compat.v1.disable_eager_execution()
m1 = Sequential()
m1.add(Embedding(86539,16,input_shape=(2527,)))
m1.add(Reshape((1,2527,16)))
m1.add(Conv2D(64,kernel_size=(16,16),padding="same"))
m1.add(LeakyReLU(alpha=0.2))
m1.add(Conv2D(4,kernel_size=(4,16),padding="same"))
m1.add(LeakyReLU(alpha=0.2))
m1.add(Flatten())
m1.add(Dense(100))
m1.add(Activation(activation='sigmoid'))
m1.add(Dense(10))
m1.add(Activation(activation='sigmoid'))
m1.add(Dense(1))
m1.add(Activation(activation='sigmoid'))
m1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

m1.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2527, 16)          1384624   
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 2527, 16)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 2527, 64)       262208    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 1, 2527, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 2527, 4)        16388     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 1, 2527, 4)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 10108)            

In [0]:
m1.fit(x=X_train,y=Y_train,epochs=5,batch_size=32,validation_data=(X_test,Y_test))

In [0]:
m1.save('imdb.h5')